In [2]:
print("fareha")

fareha


In [3]:
import numpy as np 
import pandas as pd

In [4]:
train=pd.read_csv('customer_churn_dataset-training-master.csv')
test=pd.read_csv('customer_churn_dataset-testing-master.csv')


In [5]:
train.head(2)

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0,1.0


In [6]:
test.head(2)

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,1,22,Female,25,14,4,27,Basic,Monthly,598,9,1
1,2,41,Female,28,28,7,13,Standard,Monthly,584,20,0


In [7]:
test.dropna(inplace=True)
test.reset_index(drop=True, inplace=True)
train.dropna( inplace=True)
train.reset_index(drop=True, inplace=True)


In [8]:
test.isnull().sum(),train.isnull().sum()

(CustomerID           0
 Age                  0
 Gender               0
 Tenure               0
 Usage Frequency      0
 Support Calls        0
 Payment Delay        0
 Subscription Type    0
 Contract Length      0
 Total Spend          0
 Last Interaction     0
 Churn                0
 dtype: int64,
 CustomerID           0
 Age                  0
 Gender               0
 Tenure               0
 Usage Frequency      0
 Support Calls        0
 Payment Delay        0
 Subscription Type    0
 Contract Length      0
 Total Spend          0
 Last Interaction     0
 Churn                0
 dtype: int64)

In [9]:
test.shape

(64374, 12)

In [10]:
train.shape

(440832, 12)

In [11]:
X_train=train.drop(columns=['Churn','CustomerID'],axis=1)
X_test=test.drop(columns=['Churn','CustomerID'],axis=1)
y_train=train['Churn']
y_test=test['Churn']



In [12]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((440832, 10), (64374, 10), (440832,), (64374,))

In [13]:
X_train.head(2)

,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction
0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0
1,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0


In [14]:
X_train['Subscription Type'].unique()

array(['Standard', 'Basic', 'Premium'], dtype=object)

In [15]:
X_train['Contract Length'].unique()

array(['Annual', 'Monthly', 'Quarterly'], dtype=object)

In [16]:
X_train['Gender'].unique()

array(['Female', 'Male'], dtype=object)

## pipeline

In [17]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [18]:
# data encoding transformer
#gender label
#subsciption and contract length ordinal encoding 
encoding_trf=ColumnTransformer([
    ('impute_gender', OneHotEncoder(drop='first'), [1]), 
    ('impute_subscription',OrdinalEncoder(categories=[['Basic','Standard','Premium']]),[6]),
    ('impute_contract',OrdinalEncoder(categories=[['Quarterly','Monthly','Annual']]),[7])

],remainder='passthrough')




In [19]:
# normalization
nomalization_trf=ColumnTransformer([
    ('normalized_data',StandardScaler(),[0,1,2,3,4,5,6,7,8,9])
])

In [20]:
# combining above two to form a single preprocessing pipeline 
preprocessing_pipeline=Pipeline([
    ('encoding_trf',encoding_trf),
    ('nomalization_trf',nomalization_trf)
])

In [21]:
from sklearn.linear_model import LogisticRegression
lr_pipeline=Pipeline([
    ('preprocessing_pipeline',preprocessing_pipeline),
    ('lr',LogisticRegression())
])

In [22]:
# hyperparameter tunning lr
from sklearn.model_selection import RandomizedSearchCV,StratifiedKFold
stratified_cv = StratifiedKFold(
    n_splits=5,           # Number of folds
    shuffle=True,         # Shuffle data before splitting
    random_state=42       # For reproducible results
)

param_grid={
    'lr__penalty':['l1', 'l2'],
    'lr__solver':['liblinear', 'saga'],
    'lr__C':[0.1,0.5,1,5,10,20,30,40,50]
    }
lr_clf=RandomizedSearchCV(
    estimator=lr_pipeline,
    param_distributions=param_grid,
    n_iter=100,
    scoring='f1',
    cv=stratified_cv,
    n_jobs=-1

)

In [23]:
#creating pipeline with decision tree classifier 
from sklearn.tree import DecisionTreeClassifier
dt_pipeline=Pipeline([
    ('preprocessing_pipeline',preprocessing_pipeline),
    ('dt',DecisionTreeClassifier())
])

In [34]:
#hyperparameter tunning dt
param_grid={
    'dt__criterion':['gini'],
    'dt__splitter':['best'],
    #'dt__max_depth':[3,4,5,6,7,10,20],
    #'dt__min_samples_split':[5,10,20,30,50]
}
dt_clf=RandomizedSearchCV(
    estimator=dt_pipeline,
    param_distributions=param_grid,
    n_iter=100,
    scoring='f1',
    cv=stratified_cv,
    n_jobs=-1
)

In [35]:
#training
dt_clf.fit(X_train,y_train)


c:\Users\far\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 1 is smaller than n_iter=100. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing_pipeline',
                                              Pipeline(steps=[('encoding_trf',
                                                               ColumnTransformer(remainder='passthrough',
                                                                                 transformers=[('impute_gender',
                                                                                                OneHotEncoder(drop='first'),
                                                                                                [1]),
                                                                                               ('impute_subscription',
                                                                                                OrdinalEncoder(categories=[['Basic',
                                                                                                                            'Standard',
                                                                                                                            'Premium']]),
                                                                                                [6]),
                                                                                               ('impute_contract',
                                                                                                OrdinalEncoder(categories=[['Quarterly',
                                                                                                                            'Monthly',
                                                                                                                            'Annual']]),
                                                                                                [7])])),
                                                              ('nomalization_trf',
                                                               ColumnTransformer(transformers=[('normalized_data',
                                                                                                StandardScaler(),
                                                                                                [0,
                                                                                                 1,
                                                                                                 2,
                                                                                                 3,
                                                                                                 4,
                                                                                                 5,
                                                                                                 6,
                                                                                                 7,
                                                                                                 8,
                                                                                                 9])]))])),
                                             ('dt', DecisionTreeClassifier())]),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'dt__criterion': ['gini'],
                                        'dt__splitter': ['best']},
                   scoring='f1')

In [36]:
y_pred_dt=dt_clf.predict(X_test)

In [37]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
dt_ac_score=f1_score(y_test,y_pred_dt)
dt_ac_score

0.6557850082904455

In [38]:
lr_clf.fit(X_train,y_train)

c:\Users\far\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 36 is smaller than n_iter=100. Running 36 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=Pipeline(steps=[('preprocessing_pipeline',
                                              Pipeline(steps=[('encoding_trf',
                                                               ColumnTransformer(remainder='passthrough',
                                                                                 transformers=[('impute_gender',
                                                                                                OneHotEncoder(drop='first'),
                                                                                                [1]),
                                                                                               ('impute_subscription',
                                                                                                OrdinalEncoder(categories=[['Basic',
                                                                                                                            'Standard',
                                                                                                                            'Premium']]...
                                                                                                OrdinalEncoder(categories=[['Quarterly',
                                                                                                                            'Monthly',
                                                                                                                            'Annual']]),
                                                                                                [7])])),
                                                              ('nomalization_trf',
                                                               ColumnTransformer(transformers=[('normalized_data',
                                                                                                StandardScaler(),
                                                                                                [0,
                                                                                                 1,
                                                                                                 2,
                                                                                                 3,
                                                                                                 4,
                                                                                                 5,
                                                                                                 6,
                                                                                                 7,
                                                                                                 8,
                                                                                                 9])]))])),
                                             ('lr', LogisticRegression())]),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'lr__C': [0.1, 0.5, 1, 5, 10, 20, 30,
                                                  40, 50],
                                        'lr__penalty': ['l1', 'l2'],
                                        'lr__solver': ['liblinear', 'saga']},
                   scoring='f1')

In [39]:
y_pred_lr=lr_clf.predict(X_test)

In [40]:
from sklearn.metrics import f1_score,precision_score,recall_score

In [41]:
print("logistic regression scores")
print(f1_score(y_test,y_pred_lr))
print(precision_score(y_test,y_pred_lr))
print(recall_score(y_test,y_pred_lr))


logistic regression scores
0.6927291642584672
0.53490904546834
0.9826517561407536


In [42]:
print("decision tree score")
print(f1_score(y_test,y_pred_dt))
print(precision_score(y_test,y_pred_dt))
print(recall_score(y_test,y_pred_dt))


decision tree score
0.6557850082904455
0.4881622184820069
0.9987210179385433


my model got a very high recall score but  moderate precission implies that ie
it is very good idenifying  nearly all customers who actually churned were successfully flagged but  many of the customers flagged as at risk will not actually churn.


good for buisness in which you cant afford to miss a churner 